In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import sys

# Web page scraping

In [2]:
# Create url templates for each kind of stats
per_g_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}\
_per_game.html"
adv_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}\
_advanced.html"
tot_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}\
_totals.html"
per_36m_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}_\
per_minute.html"
per_100p_url_template = "https://www.basketball-reference.com/leagues/NBA_{year}_\
per_poss.html"

# Put all the URL templates into a list
url_template_list = [per_g_url_template, adv_url_template, tot_url_template, 
                     per_36m_url_template, per_100p_url_template]

In [3]:
# Ask user to input start and end years
# Also checks to see if entry is a number
try:
    user_start_year = int(input("Enter start year in YYYY format: "))
except:
    print('Enter a valid 4 digit year.')
    
try:
    user_end_year = int(input("Enter end year in YYYY format: "))
except:
    print('Enter a valid 4 digit year.')

Enter start year in YYYY format: 1981
Enter end year in YYYY format: 2017


In [4]:
# Check if end year is after start year
if user_end_year >= user_start_year:
    print('Year range accepted.')
else:
    print('Year range is unacceptable.')

# Check if formats are in proper YYYY format
def check_year(user_input_year):
    if user_input_year > 999 and user_input_year < 10000: # Then check if it's 4 digits
        print('Year format accepted.')
    else:
        print('Enter a valid 4 digit year.')
        sys.exit()

# Check both entered years for formatting        
check_year(user_start_year)
check_year(user_end_year)

Year range accepted.
Year format accepted.
Year format accepted.


In [5]:
# Create empty lists to store data before appending to Dataframe
column_headers = []
player_data = []
# Create empty DataFrame for following functions to fill
df = pd.DataFrame()

In [6]:
# Empty DataFrames for each set of pages
df_adv = pd.DataFrame()
df_per_g = pd.DataFrame()
df_tot = pd.DataFrame()
df_per_36m = pd.DataFrame()
df_per_100p = pd.DataFrame()

# Create df_list of DataFrames for looping
df_list = [df_per_g, df_adv, df_tot, df_per_36m, df_per_100p]

In [7]:
# Get column headers from each page
# Assigns a new list of column headers each time this is called
def get_column_headers(soup):
    headers = []
    for th in soup.find('tr').findAll('th'):
        #print th.getText()
        headers.append(th.getText())
    #print headers # this line was for a bug check
    # Assign global variable to headers gathered by function
    return headers    
    #column_headers = [th.getText() for th in soup.find('tr').findAll('th')]

In [8]:
# Function to get player data from each page
def get_player_data(soup):
    # Temporary list within function to store data
    temp_player_data = []
    
    data_rows = soup.findAll('tr')[1:] # skip first row
    for i in range(len(data_rows)): # loop through each table row
        player_row = [] # empty list for each player row
        for td in data_rows[i].findAll('td'):
            player_row.append(td.getText()) # append separate data points
        temp_player_data.append(player_row) # append player row data
    return temp_player_data

In [9]:
# This function scrapes the stats data of one page and returns it in a DataFrame
def scrape_page(url):
    r = requests.get(url) # get the url
    soup = BeautifulSoup(r.text, 'html.parser') # Create BS object
    
    # call function to get column headers
    column_headers = get_column_headers(soup)
    
    # call function to get player data
    player_data = get_player_data(soup)
    
    # input data to DataFrame
    # Skip first value of column headers, 'Rk'
    df = pd.DataFrame(player_data, columns = column_headers[1:])
    
    return df

In [10]:
def get_season(input_year):
    first_yr = input_year - 1
    season = str(first_yr) + "-" + str(input_year)[2:]
    return season

In [11]:
# This function drops empty rows an columns, drops duplicates, and changes
# % character in columns
def gen_cleaning(df):
    # Convert values to numeric values first
    df = df.apply(pd.to_numeric, errors = 'ignore')
    
    # Drop columns with no data
    df.dropna(axis = 1, how = "all", inplace = True)
    
    # Drop rows with no data
    df.dropna(axis = 0, how = "all", inplace = True)
    
    # Remove duplicates player inputs; ie. players who were traded
    # I only kept the TOT per game season values
    #df.drop_duplicates(["Player"], keep = "first", inplace = True)
    
    # Change % symbol to _perc
    df.columns = df.columns.str.replace('%', '_perc')
    
    return df

In [12]:
# This function scrapes player data from multiple pages by start and end years
def scrape_pages(url_template, start_year, end_year, output_df):
    count = 0 
    for year in range(start_year, (end_year+1)):
        url = url_template.format(year = year) # grab URL per year
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html5lib') # Create soup item
        
        # Check to grab column headers
        if count == 0: # only append column headers once
            columns = get_column_headers(soup)
            count += 1
            
        # grab player data for each year
        player_data = get_player_data(soup)
        
        # Create temporary DataFrame first for each year
        # Duplicates are removed before putting into bigger DataFrame
        # These duplicates come from players playing on multiple teams in one season
        # This script only keeps the TOT output as Tm
        year_df = pd.DataFrame(player_data, columns = columns[1:])
        year_df.drop_duplicates(['Player'], keep = 'first', inplace = True)
        year_df.insert(0, 'Season', get_season(year)) # insert season year column
        
        # Append to big DataFrame for detailed cleaning
        output_df = output_df.append(year_df, ignore_index = True)
        
    # Do common, general cleaning practices
    output_df = gen_cleaning(output_df)
        
    return output_df

In [13]:
# Fill each DataFrame with data scraped from their respective pages
# Each print statement is a check for if any pages or functions give issues
# Added timer to check how long this was taking

start = time.time()

df_per_g = scrape_pages(per_g_url_template, user_start_year, user_end_year, df_per_g)
print("Finished per g")
df_adv = scrape_pages(adv_url_template, user_start_year, user_end_year, df_adv)
print("Finished adv")
df_tot = scrape_pages(tot_url_template, user_start_year, user_end_year, df_tot)
print("Finished tots")
df_per_36m = scrape_pages(per_36m_url_template, user_start_year, user_end_year, df_per_36m)
print("Finished per 36m")
df_per_100p = scrape_pages(per_100p_url_template, user_start_year, user_end_year, df_per_100p)
print("Finished per 100 possessions")

end = time.time()
print("Time elapsed :" +str((end - start) / 60) + " minutes")

Finished per g
Finished adv
Finished tots
Finished per 36m
Finished per 100 possessions
Time elapsed :14.027900115648906 minutes


# Data Auditing and Cleaning

In [14]:
# Check all column names to see what needs to be cleaned

print("totals")
print(list(df_tot))
print("per game")
print(list(df_per_g))
print("per 36 minutes")
print(list(df_per_36m))
print("advanced")
print(list(df_adv))
print("per 100p")
print(list(df_per_100p))

totals
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', '2P', '2PA', '2P_perc', 'eFG_perc', 'FT', 'FTA', 'FT_perc', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
per game
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', '2P', '2PA', '2P_perc', 'eFG_perc', 'FT', 'FTA', 'FT_perc', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PS/G']
per 36 minutes
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG_perc', '3P', '3PA', '3P_perc', '2P', '2PA', '2P_perc', 'FT', 'FTA', 'FT_perc', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
advanced
['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS_perc', '3PAr', 'FTr', 'ORB_perc', 'DRB_perc', 'TRB_perc', 'AST_perc', 'STL_perc', 'BLK_perc', 'TOV_perc', 'USG_perc', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']
per 100p
['Season', 'Player', 'Pos', 'Age', 'T

In [15]:
# Label columns properly by adding "_tot" to the end of some column values
df_tot.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]] = \
[df_tot.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]][col] + "_tot" for col in range(9)]

df_tot.columns.values[21:30] = [df_tot.columns.values[21:30][col] + \
"_tot" for col in range(9)]

In [16]:
# Check column titles again
list(df_tot)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot']

In [17]:
# drop _perc columns from per_g and per_36m
# Never mind, drop duplicates later on
# Add _per_g and _per_36m to column values
# Add _per_G to some values in df_per_g
df_per_g.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]] = \
[df_per_g.columns.values[[7, 8 , 9, 11, 12, 14, 15, 18, 19]][col] + "_per_G" for col in range(9)]

df_per_g.columns.values[21:29] = [df_per_g.columns.values[21:30][col] + \
"_per_G" for col in range(8)]

# Rename PS/G to PTS_per_G
df_per_g.rename(columns={'PS/G': 'PTS_per_G'}, inplace = True)

In [18]:
df_per_36m.columns.values[[7, 8, 9, 11, 12, 14, 15, 18, 19]]

array(['MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FTA', 'FT_perc'], dtype=object)

In [19]:
# Check if proper values were changed
list(df_per_g)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 'FG_perc',
 '3P_per_G',
 '3PA_per_G',
 '3P_perc',
 '2P_per_G',
 '2PA_per_G',
 '2P_perc',
 'eFG_perc',
 'FT_per_G',
 'FTA_per_G',
 'FT_perc',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS_per_G']

In [20]:
# Add per_36m to its column values
df_per_36m.columns.values[[8, 9, 11, 12, 14, 15, 17, 18]] = \
[df_per_36m.columns.values[[8, 9, 11, 12, 14, 15, 17, 18]][col] + "_per_36m" \
for col in range(8)]

df_per_36m.columns.values[20:30] = [df_per_36m.columns.values[20:30][col] + "_per_36m" \
                                   for col in range(9)]

In [21]:
# Check columns were changed properly
list(df_per_36m)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG_per_36m',
 'FGA_per_36m',
 'FG_perc',
 '3P_per_36m',
 '3PA_per_36m',
 '3P_perc',
 '2P_per_36m',
 '2PA_per_36m',
 '2P_perc',
 'FT_per_36m',
 'FTA_per_36m',
 'FT_perc',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m']

In [23]:
# Add per_100p to per 100 possession column values
list_of_changes = ['FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', \
                   'BLK', 'TOV', 'PF', 'PTS']
# Grab a list of current column names
column_values = list(df_per_100p.columns.values)

# Create a list for updated column names
updated_columns = []

# Loop through original column values to see what to replace
for value in column_values:
    if value in list_of_changes:
        updated_columns.append(value + '_per_100p')
    else:
        updated_columns.append(value)

# Update column values
df_per_100p.columns = updated_columns

In [24]:
# Check if columns are properly named
list(df_per_100p)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG_per_100p',
 'FGA_per_100p',
 'FG_perc',
 '3P_per_100p',
 '3PA_per_100p',
 '3P_perc',
 '2P_per_100p',
 '2PA_per_100p',
 '2P_perc',
 'FT_per_100p',
 'FTA_per_100p',
 'FT_perc',
 'ORB_per_100p',
 'DRB_per_100p',
 'TRB_per_100p',
 'AST_per_100p',
 'STL_per_100p',
 'BLK_per_100p',
 'TOV_per_100p',
 'PF_per_100p',
 'PTS_per_100p',
 'ORtg',
 'DRtg']

In [25]:
# Find where '\xa0' columns are for removal
print(df_adv.columns[-5])
print(df_adv.columns[19])

WS/48
OWS


In [26]:
# Drop '\xa0' columns, last one first
#df_adv.drop(df_adv.columns[-5], axis = 1, inplace = True)
#df_adv.drop(df_adv.columns[19], axis = 1, inplace = True)

In [27]:
list(df_adv)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'MP',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [28]:
df_adv.rename(columns = {'WS/48' : 'WS_per_48'}, inplace = True)

In [29]:
# Check to see if columns were dropped properly
list(df_adv)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'MP',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 'WS',
 'WS_per_48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [30]:
# Merge dataframes later on season, player name, and team
# Order of merges: tots, per_g, per_36m, adv
# DFs: df_tot, df_per_g, df_per_36m, df_adv
# Common things: Season, Player, Pos, Age, Tm, G

In [31]:
df_all = pd.merge(df_tot, df_per_g, how = "left", 
                 on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'FT_perc',
                      '3P_perc', '2P_perc', 'FG_perc', 'eFG_perc'])

In [32]:
df_all = pd.merge(df_all, df_per_36m, how = "left",
                 on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'FT_perc',
                      '3P_perc', '2P_perc', 'FG_perc'])

In [33]:
df_all = pd.merge(df_all, df_adv, how = "left",
                on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G'])

In [34]:
df_all = pd.merge(df_all, df_per_100p, how = "left",
             on = ['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 
                   'FG_perc', '3P_perc', '2P_perc', 'FT_perc'])

In [35]:
# Check columns to make sure they're all right
list(df_all)

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 '3P_per_G',
 '3PA_per_G',
 '2P_per_G',
 '2PA_per_G',
 'FT_per_G',
 'FTA_per_G',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS_per_G',
 'MP_x',
 'FG_per_36m',
 'FGA_per_36m',
 '3P_per_36m',
 '3PA_per_36m',
 '2P_per_36m',
 '2PA_per_36m',
 'FT_per_36m',
 'FTA_per_36m',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m',
 'MP_y',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',

In [36]:
# Try to drop duplicate MP columns
list(df_all.drop(['MP_x', 'MP_y'], axis = 1))

['Season',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP_tot',
 'FG_tot',
 'FGA_tot',
 'FG_perc',
 '3P_tot',
 '3PA_tot',
 '3P_perc',
 '2P_tot',
 '2PA_tot',
 '2P_perc',
 'eFG_perc',
 'FT_tot',
 'FTA_tot',
 'FT_perc',
 'ORB_tot',
 'DRB_tot',
 'TRB_tot',
 'AST_tot',
 'STL_tot',
 'BLK_tot',
 'TOV_tot',
 'PF_tot',
 'PTS_tot',
 'MP_per_G',
 'FG_per_G',
 'FGA_per_G',
 '3P_per_G',
 '3PA_per_G',
 '2P_per_G',
 '2PA_per_G',
 'FT_per_G',
 'FTA_per_G',
 'ORB_per_G',
 'DRB_per_G',
 'TRB_per_G',
 'AST_per_G',
 'STL_per_G',
 'BLK_per_G',
 'TOV_per_G',
 'PF_per_G',
 'PTS_per_G',
 'FG_per_36m',
 'FGA_per_36m',
 '3P_per_36m',
 '3PA_per_36m',
 '2P_per_36m',
 '2PA_per_36m',
 'FT_per_36m',
 'FTA_per_36m',
 'ORB_per_36m',
 'DRB_per_36m',
 'TRB_per_36m',
 'AST_per_36m',
 'STL_per_36m',
 'BLK_per_36m',
 'TOV_per_36m',
 'PF_per_36m',
 'PTS_per_36m',
 'PER',
 'TS_perc',
 '3PAr',
 'FTr',
 'ORB_perc',
 'DRB_perc',
 'TRB_perc',
 'AST_perc',
 'STL_perc',
 'BLK_perc',
 'TOV_perc',
 'USG_perc',
 'OWS',
 'DWS',
 

In [37]:
df_all.drop(['MP_x', 'MP_y'], axis = 1, inplace = True)

In [ ]:
# Final check of columns
list(df_all)

In [ ]:
# First check length of dataframe
print(len(df_all))

In [39]:
# Fill Null values with 0
df_all.fillna(0, inplace = True)

In [40]:
# Address ambiguous positions and combination positions
df = df_all.groupby(['Pos'])['Pos'].nunique()
df

Pos
0        1
C        1
C-PF     1
C-SF     1
PF       1
PF-C     1
PF-SF    1
PG       1
PG-SF    1
PG-SG    1
SF       1
SF-PF    1
SF-SG    1
SG       1
SG-PF    1
SG-PG    1
SG-SF    1
Name: Pos, dtype: int64

In [41]:
# Remove where 'Pos' value is 0
df_all = df_all[df_all['Pos'] != 0]

# Then check df_all length again
print(len(df_all))

15234


In [42]:
# I think the PG-SF and C-SF positions are mistakes
# Check the value to see the player
df_all[df_all['Pos'] == 'C-SF']

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
10817,2007-08,Bobby Jones,C-SF,24.0,TOT,47.0,2.0,531.0,60.0,140.0,...,6.0,8.7,2.7,1.4,0.3,2.7,8.0,16.9,106.0,110.0


In [43]:
# Check Bobby Jones' actual, commonly played position
df_all[df_all['Player'] == 'Bobby Jones']

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
142,1980-81,Bobby Jones,PF,29.0,PHI,81.0,0.0,2046.0,407.0,755.0,...,6.6,9.9,5.1,2.2,1.7,3.4,5.1,24.9,118.0,99.0
448,1981-82,Bobby Jones,PF,30.0,PHI,76.0,73.0,2181.0,416.0,737.0,...,6.2,8.6,4.1,2.2,2.5,3.2,4.6,24.0,117.0,103.0
756,1982-83,Bobby Jones,PF,31.0,PHI,74.0,0.0,1749.0,250.0,460.0,...,6.5,9.2,3.8,2.3,2.4,2.9,5.3,17.8,117.0,99.0
1067,1983-84,Bobby Jones,PF,32.0,PHI,75.0,0.0,1761.0,226.0,432.0,...,6.3,8.8,5.1,2.9,2.8,2.8,5.4,16.9,116.0,101.0
1377,1984-85,Bobby Jones,PF,33.0,PHI,80.0,8.0,1633.0,207.0,385.0,...,5.6,8.7,4.5,2.4,1.5,3.4,5.3,17.5,118.0,106.0
1698,1985-86,Bobby Jones,PF,34.0,PHI,70.0,42.0,1519.0,189.0,338.0,...,3.8,5.3,4.0,1.5,1.6,2.8,5.0,15.5,114.0,108.0
10354,2006-07,Bobby Jones,SF,23.0,PHI,44.0,5.0,336.0,43.0,93.0,...,5.5,9.0,2.5,1.7,0.2,2.8,9.3,17.3,99.0,108.0
10817,2007-08,Bobby Jones,C-SF,24.0,TOT,47.0,2.0,531.0,60.0,140.0,...,6.0,8.7,2.7,1.4,0.3,2.7,8.0,16.9,106.0,110.0


In [44]:
# Create list of dual positions in DataFrame
# Create empty DataFrame to audit dual position values
column_names = list(df_all.columns.values)
dual_pos_rows = []
df_dual_pos = pd.DataFrame(columns = column_names)

In [45]:
# Gather all the dual positions by seeing which ones have a dash
for pos in df_all['Pos']:
    if "-" in pos:
        if pos not in dual_pos_rows:
            dual_pos_rows.append(pos)

In [46]:
# Append all dual position rows to a new DataFrame for auditing
for pos in dual_pos_rows:
    df_dual_pos = df_dual_pos.append(df_all[df_all['Pos'] == pos],
                                    ignore_index = True)

In [47]:
df_dual_pos
# It looks like all these players moved teams before
# Certain players have multiple positions or changed positions

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
0,1980-81,Terry Duerod,SG-SF,24.0,TOT,50.0,0.0,451.0,104.0,234.0,...,2.9,4.7,3.9,1.8,0.4,3.8,2.9,26.5,97.0,110.0
1,1990-91,Tony Brown,SG-SF,30.0,TOT,30.0,0.0,294.0,30.0,80.0,...,3.3,7.4,2.7,0.7,0.0,2.7,8.1,14.2,97.0,110.0
2,1995-96,Michael Curry,SG-SF,27.0,TOT,46.0,1.0,783.0,73.0,161.0,...,4.0,5.9,1.9,1.7,0.1,1.7,6.4,14.7,112.0,107.0
3,1996-97,Jim Jackson,SG-SF,26.0,TOT,77.0,77.0,2831.0,444.0,1029.0,...,5.2,7.7,5.9,1.6,0.6,3.9,3.6,23.0,106.0,111.0
4,1997-98,Aaron McKie,SG-SF,25.0,TOT,81.0,32.0,1813.0,139.0,381.0,...,5.1,6.8,5.2,3.0,0.4,2.2,4.8,9.8,92.0,104.0
5,1997-98,Jerry Stackhouse,SG-SF,23.0,TOT,79.0,37.0,2545.0,424.0,975.0,...,3.4,5.6,5.1,1.9,1.3,4.8,3.7,26.5,103.0,106.0
6,1998-99,Rodrick Rhodes,SG-SF,25.0,TOT,13.0,1.0,156.0,13.0,52.0,...,2.8,5.9,3.8,1.7,0.7,6.9,7.3,14.9,67.0,110.0
7,2002-03,Gordan Giricek,SG-SF,25.0,TOT,76.0,62.0,2148.0,350.0,803.0,...,4.9,5.7,3.3,1.3,0.2,3.5,3.8,22.5,99.0,109.0
8,2003-04,DeShawn Stevenson,SG-SF,22.0,TOT,80.0,78.0,2444.0,376.0,871.0,...,4.9,6.6,3.5,1.2,0.4,2.7,3.1,20.2,98.0,109.0
9,2004-05,Casey Jacobsen,SG-SF,23.0,TOT,84.0,1.0,1798.0,165.0,408.0,...,3.9,4.9,3.3,1.0,0.2,2.1,3.0,16.0,111.0,112.0


In [48]:
df_dual_pos.groupby(['Player']).size().reset_index(name = 'Count').sort_values(['Count'], ascending = False).head(n=10)

,Player,Count
5,Allen Iverson*,3
25,Brian Skinner,2
143,Ricky Davis,2
109,Lonny Baxter,2
76,Jamie Feick,2
22,Bostjan Nachbar,2
84,Jeremy Richardson,2
105,Larry Hughes,2
171,Tony Brown,2
159,Stephen Jackson,2


In [49]:
# Check what is going on with some players with multiple positions
df_all[df_all['Player'] == 'Allen Iverson*']

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
5870,1996-97,Allen Iverson*,PG,21.0,PHI,76.0,74.0,3045.0,625.0,1504.0,...,3.3,5.2,9.4,2.6,0.4,5.6,3.9,29.6,104.0,112.0
6320,1997-98,Allen Iverson*,PG,22.0,PHI,80.0,80.0,3150.0,649.0,1407.0,...,3.5,5.0,8.3,3.0,0.4,4.1,3.4,29.6,109.0,106.0
6751,1998-99,Allen Iverson*,SG,23.0,PHI,48.0,48.0,1990.0,435.0,1056.0,...,4.6,6.4,6.1,3.0,0.2,4.5,2.7,34.9,105.0,99.0
7201,1999-00,Allen Iverson*,SG,24.0,PHI,70.0,70.0,2853.0,729.0,1733.0,...,3.6,4.8,6.0,2.6,0.1,4.2,2.9,36.1,101.0,102.0
7640,2000-01,Allen Iverson*,SG,25.0,PHI,71.0,71.0,2979.0,762.0,1813.0,...,4.0,4.9,5.8,3.2,0.4,4.2,2.6,39.3,106.0,99.0
8087,2001-02,Allen Iverson*,SG,26.0,PHI,60.0,59.0,2622.0,665.0,1669.0,...,4.6,5.5,6.8,3.5,0.3,4.9,2.1,38.8,101.0,100.0
8518,2002-03,Allen Iverson*,SG,27.0,PHI,82.0,82.0,3485.0,804.0,1940.0,...,4.2,5.2,6.8,3.4,0.2,4.3,2.2,34.1,103.0,102.0
8961,2003-04,Allen Iverson*,SG,28.0,PHI,48.0,47.0,2040.0,435.0,1125.0,...,3.8,4.8,8.7,3.1,0.1,5.6,2.3,33.8,96.0,102.0
9414,2004-05,Allen Iverson*,PG,29.0,PHI,75.0,75.0,3174.0,771.0,1818.0,...,3.9,4.8,9.5,2.9,0.1,5.5,2.2,36.7,105.0,105.0
9871,2005-06,Allen Iverson*,PG,30.0,PHI,72.0,72.0,3103.0,815.0,1822.0,...,3.1,3.9,8.9,2.3,0.2,4.1,2.0,39.7,111.0,110.0


In [50]:
# Find most common position for this player
df_all[df_all['Player'] == 'Allen Iverson*']\
.groupby(['Pos']).size().reset_index(name = 'Count')\
.sort_values(['Count'], ascending = False).iloc[0][0]

'SG'

In [51]:
# Count of seasons played at most common position
df_all[df_all['Player'] == 'Allen Iverson*'].groupby(['Pos']).size().iloc[0]

4

In [52]:
# Use dictionary as key to replace 'Pos' values in the big DataFrame
most_common_pos = {}
# Saves a dictionary of player names with equally common positions
two_common_pos = {}
# PG, SG, SF, PF, C are 1-5, respectively
pos_key = {'PG': '1', 'SG': '2', 'SF': '3', 'PF': '4', 'C': '5'}

# Side note: This makes Tim Duncan a center

def grab_most_common_pos(df, pos_dict):
    # Loop through a dataframe and assign names and most common positions to a dictionary
    for index, row in df[['Player', 'Pos']].iterrows():
        player_name = row.iloc[0] # Assign player name to variable
        # subset position dataframe to a player
        pos_df = df[df['Player'] == player_name].groupby('Pos').size()\
        .reset_index(name = 'Count')\
        .sort_values(['Count'], ascending = False) 
        
        #dual_pos_dict = {} # Store dict of dual positions
        
        pos = pos_df.iloc[0][0] # Assign first position to variable
        second_pos = '' 
        
        # Fill in second position if it exists
        if len(pos_df) > 1:
            second_pos = pos_df.iloc[1][0]    
            
        # Check is player has a second common position
        # I don't know what to do in this situation yet
        #if pos_df.iloc[0][1] == pos_df.iloc[1][1]:
        #    dual_pos_dict['First position'] = pos
        #    dual_pos_dict['Second position'] = second_pos
        #    two_common_pos[player_name] = dual_pos_dict
        #print(player_name)
        
        if player_name not in pos_dict.keys(): # Check if name exists first
            pos_dict[player_name] = pos
    
    #return pos_dict

def clean_pos(df, pos_dict):
    # Loop through rows to check players' positions
    grab_most_common_pos(df, pos_dict)
    
    # If the most common position is a dual position, take the first one
    for name, pos in pos_dict.items():
        if '-' in pos:
            index = pos.find('-')
            pos_dict[name] = pos[:index] 
        else:
            continue
    
    # Change pos_dict values to 1-5 from key
    for key, value in pos_dict.items():
        pos_dict[key] = pos_key[value]
    
    # Return DataFrame with cleaned positions
    return df

In [53]:
# This function takes in a DataFrame and adds a new column with Rounded Position values
def assign_pos(df, pos_dict):    
    # Add a Rounded_Pos column and fill it from pos_dict
    df['Rounded_Pos'] = ''
    
    for name, pos in pos_dict.items():
        df.Rounded_Pos[df['Player'] == name] = pos

In [54]:
clean_pos(df_all, most_common_pos)

,Season,Player,Pos,Age,Tm,G,GS,MP_tot,FG_tot,FGA_tot,...,DRB_per_100p,TRB_per_100p,AST_per_100p,STL_per_100p,BLK_per_100p,TOV_per_100p,PF_per_100p,PTS_per_100p,ORtg,DRtg
0,1980-81,Kareem Abdul-Jabbar*,C,33.0,LAL,80.0,0.0,2976.0,836.0,1457.0,...,9.8,12.9,4.3,0.9,3.6,3.9,3.8,32.9,118.0,101.0
1,1980-81,Tom Abernethy,SF,26.0,TOT,39.0,0.0,298.0,25.0,59.0,...,4.5,7.7,3.0,1.1,0.5,1.3,5.4,10.1,107.0,109.0
2,1980-81,Alvan Adams,C,26.0,PHO,75.0,0.0,2054.0,458.0,870.0,...,8.7,12.2,7.7,2.4,1.5,5.0,5.0,24.9,108.0,96.0
3,1980-81,Darrell Allums,PF,22.0,DAL,22.0,0.0,276.0,23.0,67.0,...,8.1,11.5,4.4,0.9,1.4,4.1,9.0,10.4,79.0,109.0
4,1980-81,Tiny Archibald*,PG,32.0,BOS,80.0,72.0,2820.0,382.0,766.0,...,2.4,3.0,10.4,1.3,0.3,4.5,3.4,18.7,111.0,107.0
5,1980-81,Dennis Awtrey,C,32.0,SEA,47.0,0.0,607.0,44.0,93.0,...,5.9,8.5,4.3,0.9,0.6,2.6,6.7,8.0,98.0,106.0
6,1980-81,James Bailey,PF,23.0,SEA,82.0,0.0,2539.0,444.0,889.0,...,7.8,11.4,1.8,1.4,2.7,4.1,6.3,21.6,101.0,102.0
7,1980-81,Greg Ballard,SF,26.0,WSB,82.0,0.0,2610.0,549.0,1186.0,...,7.4,10.4,3.5,2.1,0.7,2.1,3.5,22.8,105.0,101.0
8,1980-81,Mike Bantom,SF,29.0,IND,76.0,0.0,2375.0,431.0,882.0,...,5.5,8.5,4.8,1.6,1.7,3.9,5.7,21.2,103.0,105.0
9,1980-81,Billy Ray Bates,SG,24.0,POR,77.0,0.0,1560.0,439.0,902.0,...,2.6,4.7,5.9,2.4,0.2,4.5,3.6,31.7,106.0,107.0


In [55]:
assign_pos(df_all, most_common_pos)

/Users/benjaminxiao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


# Write to csv file for use

In [56]:
# Create a DataFrame with top 25 single season scorers 
#df_top_25_scorers = df_all.sort_values('PTS_per_G', ascending = False).head(n=25)

# Create a DataFrame with top 50 single season scorers 
#df_top_50_scorers = df_all.sort_values('PTS_per_G', ascending = False).head(n=50)

In [57]:
# Write to CSV files and DONE!
file_name = 'player_data_' + str(user_start_year) + '-' + str(user_end_year) + '.csv'
#print(file_name)
df_all.to_csv(file_name, encoding = 'utf-8', index = False)

In [58]:
#df_top_50_scorers.to_csv("bref_1981_2017_top_50_season_scorers.csv", encoding = "utf-8", index = False)